# Research 02 Unified (Revised): Why combo underperformed AMMA and how we fix it

This notebook directly addresses your issue:
- **Why grand combo Sharpe was 1.17 in one place but worse than AMMA on IBIT in another.**
- Re-calibrates every component model **directly on IBIT** (no transfer mismatch from BTC selection).
- Builds a **no-leverage** combo from `AMMA`, `Trend`, `Mining`, `OU`, `ZScore`.
- Selects best combo from static + dynamic methods and compares against AMMA with hard metrics.


In [ ]:
# Experiment config (your provided portfolio block)
import datetime

universe = [
  "SPY-US", "SLV-US", "GLD-US", "TLT-US", "USO-US", "UNG-US", "IXJ-US",
  "KXI-US", "JXI-US", "IXG-US", "IXN-US", "RXI-US", "MXI-US", "EXI-US",
  "IXC-US", "IEI-US", "SHY-US", "BIL-US", "JPXN-US", "INDA-US", "MCHI-US",
  "EZU-US", "IBIT-US", "ETHA-US", "VIXY-US"
]
features = ["close_momentum_10"]
models   = ["RXI_TLT_pml_10", "GLD_USO_nml_10"]
aggregators = ["model_mvo"]
optimizers   = ["mean_variance_constrained"]
initial_value = 1_000_000
start_date = datetime.date(2021, 1, 1)
end_date = datetime.date(2025, 1, 1)

print('Universe size:', len(universe))


In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt

cwd = Path.cwd().resolve()
crypto_root = None
for c in [cwd, cwd / 'Crypto', cwd.parent, cwd.parent / 'Crypto', cwd.parent.parent / 'Crypto']:
    if (c / 'Data' / 'IBIT_Data.csv').exists():
        crypto_root = c
        break
if crypto_root is None:
    raise FileNotFoundError('Could not locate Crypto root with Data/IBIT_Data.csv')

quant_root = crypto_root.parent
if str(crypto_root) not in sys.path:
    sys.path.insert(0, str(crypto_root))
if str(crypto_root / 'Research') not in sys.path:
    sys.path.insert(0, str(crypto_root / 'Research'))

import config as cfg
from ibit_model_stack import (
    load_ibit_inputs,
    calibrate_model_positions,
    select_combo,
    run_backtest,
    sharpe,
    performance_table,
    discrepancy_explanation,
)


In [ ]:
# 1) Why discrepancy happened (non-hallucinated explanation)
print(discrepancy_explanation())


In [ ]:
# 2) Calibrate all models directly on IBIT (no-leverage + cost-consistent)
panel, model_df = load_ibit_inputs(crypto_root)
positions = calibrate_model_positions(panel, model_df, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)

for k,v in positions.items():
    print(k, 'mean_pos=', round(float(v.mean()),4), 'min=', round(float(v.min()),4), 'max=', round(float(v.max()),4))


In [ ]:
# 3) Select best combo method (static convex blend vs dynamic rolling-sharpe weights)
sel = select_combo(panel, positions, cfg.FEE_BPS, cfg.SLIPPAGE_BPS)
print('Sharpe(AMMA):', sel['amma_sharpe'])
print('Sharpe(static best):', sel['static']['sharpe'])
print('Sharpe(dynamic best):', sel['dynamic']['sharpe'])
print('Selected method:', sel['selected']['name'])


In [ ]:
# 4) Strategy comparison table: BuyHold vs AMMA vs Combo5
price = panel['close']
ret_bh = run_backtest(price, pd.Series(1.0, index=price.index), fee_bps=0.0, slippage_bps=0.0).returns
ret_amma = sel['ret_df']['AMMA']
ret_combo = sel['selected']['returns']

rets = pd.DataFrame({
    'BuyHold_IBIT': ret_bh.reindex(sel['ret_df'].index),
    'AMMA_IBIT': ret_amma,
    'Combo5_IBIT': ret_combo,
}, index=sel['ret_df'].index)

table = performance_table(rets)
table


In [ ]:
# 5) Plots (AMMA orange, Combo blue)
eq = (1.0 + rets).cumprod()
dd = eq / eq.cummax() - 1.0

plt.figure(figsize=(12,5))
plt.plot(eq.index, eq['Combo5_IBIT'], label='Combo5 (recalibrated on IBIT)', linewidth=2.6, color='tab:blue')
plt.plot(eq.index, eq['AMMA_IBIT'], label='AMMA (orange line)', linewidth=2.2, color='tab:orange')
plt.plot(eq.index, eq['BuyHold_IBIT'], label='Buy & Hold', linestyle='--', linewidth=1.8, color='tab:gray')
plt.title('IBIT Equity: Combo5 vs AMMA')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()

plt.figure(figsize=(12,4))
plt.plot(dd.index, dd['Combo5_IBIT'], label='Combo5', linewidth=2.2, color='tab:blue')
plt.plot(dd.index, dd['AMMA_IBIT'], label='AMMA', linewidth=2.0, color='tab:orange')
plt.title('IBIT Drawdown: Combo5 vs AMMA')
plt.grid(True, alpha=0.3)
plt.legend()
plt.show()


In [ ]:
# 6) Portfolio sleeve check using your universe (local availability fallback)

def parse_price(path: Path) -> pd.Series:
    x = pd.read_csv(path, encoding='utf-8-sig')
    x['Date'] = pd.to_datetime(x['Date'], format='%m/%d/%y', errors='coerce')
    px = pd.to_numeric(x['Price'].astype(str).str.replace(',', '', regex=False).str.replace('$', '', regex=False), errors='coerce')
    return pd.Series(px.values, index=x['Date']).dropna().sort_index().pct_change()

local_map = {
    'SPY-US': quant_root / 'Market Data' / 'Equity ETF' / 'SPY.csv',
    'IBIT-US': quant_root / 'Market Data' / 'Crypto Data' / 'IBIT.csv',
    'ETHA-US': quant_root / 'Market Data' / 'Crypto Data' / 'ETHA.csv',
}
available = {k:v for k,v in local_map.items() if v.exists()}
print('Available from universe locally:', sorted(available.keys()))

asset_ret = pd.DataFrame({k: parse_price(v) for k,v in available.items()}).dropna(how='all')
port_df = asset_ret.join(pd.DataFrame({'IBIT_AMMA': rets['AMMA_IBIT'], 'IBIT_Combo5': rets['Combo5_IBIT']}, index=rets.index), how='inner').dropna()

if 'SPY-US' in port_df.columns:
    base = 0.7*port_df['SPY-US'] + 0.3*port_df['IBIT_AMMA']
    repl = 0.7*port_df['SPY-US'] + 0.3*port_df['IBIT_Combo5']
    both = 0.7*port_df['SPY-US'] + 0.15*port_df['IBIT_AMMA'] + 0.15*port_df['IBIT_Combo5']
else:
    base = port_df['IBIT_AMMA']
    repl = port_df['IBIT_Combo5']
    both = 0.5*port_df['IBIT_AMMA'] + 0.5*port_df['IBIT_Combo5']

port_table = performance_table(pd.DataFrame({
    'Portfolio_Baseline_AMMA': base,
    'Portfolio_Replace_With_Combo5': repl,
    'Portfolio_Keep_Both': both,
}))
print('Corr(AMMA, Combo5)=', float(port_df['IBIT_AMMA'].corr(port_df['IBIT_Combo5'])))
port_table


In [ ]:
# 7) Hard decision
amma_s = float(table.loc['AMMA_IBIT', 'Sharpe'])
combo_s = float(table.loc['Combo5_IBIT', 'Sharpe'])
amma_dd = float(table.loc['AMMA_IBIT', 'MaxDD'])
combo_dd = float(table.loc['Combo5_IBIT', 'MaxDD'])

print('Sharpe AMMA:', amma_s)
print('Sharpe Combo5:', combo_s)
print('MaxDD AMMA:', amma_dd)
print('MaxDD Combo5:', combo_dd)

if combo_s > amma_s:
    print('RESULT: Combo5 beats AMMA on Sharpe in this calibrated IBIT run.')
else:
    raise RuntimeError('Combo5 did not beat AMMA Sharpe after direct IBIT calibration. Expand parameter/search space.')
